In [ ]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.2'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-02-24 01:51:11--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.28MB/s    in 0.7s    

2021-02-24 01:51:12 (1.28 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://aws-gt-finalproject001.s3.us-east-2.amazonaws.com/full_esrb_rating_data.csv"
spark.sparkContext.addFile(url)
user_data_df = spark.read.csv(SparkFiles.get("full_esrb_rating_data.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
user_data_df.show()

+---+--------------------+-------+-----------------+--------------+-----+--------------+----------------+-----------+--------------+----------------+----------------+--------+------+------------+----------+---------------------+---------------------+-------------+-----------+----------------------+-------------+--------------+------+--------------+--------------+-------------+------------------+---------------+---------------------+-----------------+--------------+------------------------+--------+-----------+
|_c0|               title|console|alcohol_reference|animated_blood|blood|blood_and_gore|cartoon_violence|crude_humor|drug_reference|fantasy_violence|intense_violence|language|lyrics|mature_humor|mild_blood|mild_cartoon_violence|mild_fantasy_violence|mild_language|mild_lyrics|mild_suggestive_themes|mild_violence|no_descriptors|nudity|partial_nudity|sexual_content|sexual_themes|simulated_gambling|strong_language|strong_sexual_content|suggestive_themes|use_of_alcohol|use_of_drugs_an

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from matplotlib import style
style.use("ggplot")
# from matplotlib import rcParams
# rcParams['figure.figsize'] = 10, 8

from sklearn.neighbors import KNeighborsClassifier
import pandas as pd

import os

In [ ]:
# user_data_df = pd.read_csv(os.path.join("..", "full_esrb_rating_data.csv"))
# user_data_df.head()
user_data_df.show()

+---+--------------------+-------+-----------------+--------------+-----+--------------+----------------+-----------+--------------+----------------+----------------+--------+------+------------+----------+---------------------+---------------------+-------------+-----------+----------------------+-------------+--------------+------+--------------+--------------+-------------+------------------+---------------+---------------------+-----------------+--------------+------------------------+--------+-----------+
|_c0|               title|console|alcohol_reference|animated_blood|blood|blood_and_gore|cartoon_violence|crude_humor|drug_reference|fantasy_violence|intense_violence|language|lyrics|mature_humor|mild_blood|mild_cartoon_violence|mild_fantasy_violence|mild_language|mild_lyrics|mild_suggestive_themes|mild_violence|no_descriptors|nudity|partial_nudity|sexual_content|sexual_themes|simulated_gambling|strong_language|strong_sexual_content|suggestive_themes|use_of_alcohol|use_of_drugs_an

In [ ]:
# user_data_df.esrb_rating.unique()
user_data_df.toPandas()['esrb_rating'].unique()


array(['E', 'ET', 'M', 'T'], dtype=object)

In [ ]:
target = user_data_df["esrb_rating"]
target_names = ["E", "ET", "M", "T"]

In [ ]:
data = user_data_df.drop("Unnamed: 0","title", "console", "esrb_rating")
# feature_names = data.columns
# feature_names.show()
data.show()

+---+-----------------+--------------+-----+--------------+----------------+-----------+--------------+----------------+----------------+--------+------+------------+----------+---------------------+---------------------+-------------+-----------+----------------------+-------------+--------------+------+--------------+--------------+-------------+------------------+---------------+---------------------+-----------------+--------------+------------------------+--------+
|_c0|alcohol_reference|animated_blood|blood|blood_and_gore|cartoon_violence|crude_humor|drug_reference|fantasy_violence|intense_violence|language|lyrics|mature_humor|mild_blood|mild_cartoon_violence|mild_fantasy_violence|mild_language|mild_lyrics|mild_suggestive_themes|mild_violence|no_descriptors|nudity|partial_nudity|sexual_content|sexual_themes|simulated_gambling|strong_language|strong_sexual_content|suggestive_themes|use_of_alcohol|use_of_drugs_and_alcohol|violence|
+---+-----------------+--------------+-----+------

# Training The Data

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=42)

TypeError: ignored

In [ ]:
X_train

,alcohol_reference,animated_blood,blood,blood_and_gore,cartoon_violence,crude_humor,drug_reference,fantasy_violence,intense_violence,language,...,partial_nudity,sexual_content,sexual_themes,simulated_gambling,strong_janguage,strong_sexual_content,suggestive_themes,use_of_alcohol,use_of_drugs_and_alcohol,violence
2295,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
785,0,0,1,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
940,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2275,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
8,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1638,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1095,0,0,0,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
1130,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1294,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
y_train

2295    ET
785      M
940      M
2275    ET
8       ET
        ..
1638     E
1095    ET
1130     T
1294     T
860     ET
Name: esrb_rating, Length: 1796, dtype: object

In [ ]:
# Support vector machine linear classifier
from sklearn.svm import SVC 
model = SVC(kernel='linear')


In [ ]:
model.fit(X_train, y_train)

SVC(kernel='linear')

In [ ]:
# Model Accuracy
print('Test Acc: %.3f' % model.score(X_test, y_test))

Test Acc: 0.841


In [ ]:
# Calculate classification report
from sklearn.metrics import classification_report
predictions = model.predict(X_test)
print(classification_report(y_test, predictions,
                            target_names=target_names))

              precision    recall  f1-score   support

           E       0.96      0.98      0.97       132
          ET       0.71      0.81      0.76       122
           M       0.83      0.89      0.86       111
           T       0.85      0.75      0.80       234

    accuracy                           0.84       599
   macro avg       0.84      0.86      0.85       599
weighted avg       0.85      0.84      0.84       599



# Saving Trained Model

In [ ]:
# import sklearn.externals.joblib as extjoblib
import joblib

In [ ]:
# Save the model
joblib_file = "joblib_model.pk1"
joblib.dump(model, joblib_file) 


['joblib_model.pk1']